TODO: Figure out ravel() 1d array problem

In [2]:
import pandas as pd
import numpy as np
import os
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from __future__ import division
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
labels = pd.read_csv('stocknews//labels.csv', header=None)

http://stackoverflow.com/questions/28382735/python-pandas-does-not-read-the-first-row-of-csv-file

`pd.read_csv` was cutting off the first row of labels

In [4]:
# Confirm size of labels to make sure data loaded correctly
labels.shape

(1989, 1)

In [5]:
str_tokens = pd.read_csv('stocknews//tokens_str.csv', dtype=str, keep_default_na=False)

In [6]:
str_tokens.head()

,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,georgia downs two russian warplanes countries ...,breaking musharraf impeached,russia today columns troops roll south ossetia...,russian tanks moving towards capital south oss...,afghan children raped impunity official says s...,150 russian tanks entered south ossetia whilst...,breaking georgia invades south ossetia russia ...,enemy combatent trials nothing sham salim hama...,georgian troops retreat osettain capital presu...,prep georgia war russia,...,georgia invades south ossetia russia gets invo...,al qaeda faces islamist backlash,condoleezza rice us would act prevent israeli ...,busy day european union approved new sanctions...,georgia withdraw 000 soldiers iraq help fight ...,pentagon thinks attacking iran bad idea us new...,caucasus crisis georgia invades south ossetia,indian shoe manufactory series like work,visitors suffering mental illnesses banned oly...,help mexico kidnapping surge
1,wont america nato help us wont help us help iraq,bush puts foot georgian conflict,jewish georgian minister thanks israeli traini...,georgian army flees disarray russians advance ...,olympic opening ceremony fireworks faked,mossad fraudulent new zealand passports iraq,russia angered israeli military sale georgia,american citizen living ossetia blames georgia...,welcome world war iv high definition,georgia move mistake monumental proportions,...,israel us behind georgian aggression,believe tv neither russian georgian much victims,riots still going montreal canada police murde...,china overtake us largest manufacturer,war south ossetia pics,israeli physicians group condemns state torture,russia beaten united states head peak oil,perhaps question georgia russia conflict,russia much better war,come trading sex food
2,remember adorable year old sang opening ceremo...,russia ends georgia operation,sexual harassment would children,al qa eda losing support iraq brutal crackdown...,ceasefire georgia putin outmaneuvers west,microsoft intel tried kill xo 100 laptop,stratfor russo georgian war balance power,trying get sense whole georgia russia war vote...,us military surprised timing swiftness russian...,beats war drum iran dumps dollar,...,troops still georgia know georgia first place,russias response georgia right,gorbachev accuses making serious blunder pursu...,russia georgia nato cold war two,remember adorable 62 year old led country war ...,war georgia israeli connection,signs point us encouraging georgia invade sout...,christopher king argues us nato behind georgia...,america new mexico,bbc news asia pacific extinction man climate
3,refuses israel weapons attack iran report,president ordered attack tskhinvali capital so...,israel clears troops killed reuters cameraman,britain policy tough drugs pointless says form...,body 14 year old found trunk latest ransom pai...,china moved 10 million quake survivors prefab ...,bush announces operation get russia grill yeah...,russian forces sink georgian ships,commander navy air reconnaissance squadron pro...,92 cnn readers russia actions georgia justified,...,elephants extinct 2020,us humanitarian missions soon georgia russia h...,georgia ddos came us sources,russian convoy heads georgia violating truce,israeli defence minister us strike iran,gorbachev choice,witness russian forces head towards tbilisi br...,quarter russians blame conflict poll,georgian president says us military take contr...,2006 nobel laureate aleksander solzhenitsyn ac...
4,experts admit legalise drugs,war south osetia 89 pictures made russian soldier,swedish wrestler ara abrahamian throws away me...,russia exaggerated death toll south ossetia 44...,missile killed inside pakistan may launched cia,rushdie condemns random house refusal publish ...,poland us agree missle defense deal interestin...,russians conquer tblisi bet seriously bet,russia exaggerating south ossetian death toll ...,musharraf expected resign rathe

In [7]:
str_tokens.shape

(1989, 25)

In [8]:
def replace_num(element):
    return ' '.join([('numero' if k.isdigit() else k) for k in element.split()])

http://stackoverflow.com/questions/6905636/python-conditional-list-joins

Instead of a digit, use `'NUMERO'` as it is resistant to stemming/lemmatizing.  It's possible that headlines with numbers might contain some information.  This will prevent preprocessing from discarding the information.

In [9]:
str_tokens['merged'] = str_tokens.iloc[:, 0:].apply(lambda x: ' '.join(x.dropna().values.tolist()), axis=1)

In [10]:
# type(str_tokens['merged'][0])

In [11]:
str_tokens['merged'] = str_tokens['merged'].apply(replace_num)

In [12]:
from nltk.stem.porter import PorterStemmer

In [13]:
pstem = PorterStemmer()

In [14]:
def stm(element):
    return ' '.join([pstem.stem(k) for k in element.split()])

In [15]:
str_tokens['merged'] = str_tokens['merged'].apply(stm)

In [16]:
str_tokens['merged']

0       georgia down two russian warplan countri move ...
1       wont america nato help us wont help us help ir...
2       rememb ador year old sang open ceremoni fake r...
3       refus israel weapon attack iran report presid ...
4       expert admit legalis drug war south osetia num...
5       mom miss gay man bad numero year old cheerlead...
6       afghan prison major femal prison serv numero y...
7       man arrest lock five hour take photo polic van...
8       two elderli chines women sentenc year educ lab...
9       british resid held guantanamo bay win legal ba...
10      syria say readi put russian missil system soil...
11      korea kim die numero replac lookalik say wased...
12      north korea halt denuclearis us fail remov lis...
13      photo numero year old iraqi suicid bomber gave...
14      militari help georgia declar war say moscow we...
15      russian prime minist vladimir putin accus us p...
16      girl film isra soldier brutal sinc famili vict...
17      poland

Split labels and tokens into 4 folds
Fold 1 = [:499]
Fold 2 = [499:996]
Fold 3 = [996:1493]
Fold 4 = [1493:]

## Split data into train and test sets

In [17]:
train_text = str_tokens.merged[0:1493] # train features

In [18]:
test_text = str_tokens.merged[1493:] # test features

In [19]:
train_labels = labels[0:1493].values # train labels

In [20]:
test_labels = labels[1493:].values; # test labels

In [21]:
vectorizer = TfidfVectorizer( max_features = 40000, ngram_range = ( 1, 1 ), 
sublinear_tf = True )

In [22]:
# Only need text, not labels
train_x = vectorizer.fit_transform( train_text )

In [23]:
test_x = vectorizer.transform( test_text )

In [24]:
train_x.shape

(1493, 19122)

In [25]:
test_x.shape

(496, 19122)

In [26]:
from sklearn.linear_model import PassiveAggressiveClassifier

In [27]:
classifier = PassiveAggressiveClassifier(n_iter=8)

In [28]:
train_labels.shape

(1493, 1)

In [29]:
train_labels.ravel()

array([0, 1, 0, ..., 1, 1, 1])

In [30]:
classifier.fit(train_x, train_labels.ravel())

PassiveAggressiveClassifier(C=1.0, class_weight=None, fit_intercept=True,
              loss='hinge', n_iter=8, n_jobs=1, random_state=None,
              shuffle=True, verbose=0, warm_start=False)

In [31]:
classifier.score(test_x, test_labels.ravel())

0.51008064516129037

In [32]:
from sklearn.linear_model import SGDClassifier

In [33]:
classifier = PassiveAggressiveClassifier(loss='log', n_iter=8)

In [34]:
classifier.fit(train_x, train_labels.ravel())

PassiveAggressiveClassifier(C=1.0, class_weight=None, fit_intercept=True,
              loss='log', n_iter=8, n_jobs=1, random_state=None,
              shuffle=True, verbose=0, warm_start=False)

In [35]:
classifier.score(test_x, test_labels.ravel())

0.49596774193548387

### Ridge Classifier

In [36]:
from sklearn.linear_model import RidgeClassifier

In [37]:
clf = RidgeClassifier()

In [38]:
clf.fit(train_x, train_labels.ravel())

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='auto',
        tol=0.001)

In [39]:
clf.score(test_x, test_labels.ravel())

0.48790322580645162

### Gaussian Naive Bayes

In [40]:
from sklearn.naive_bayes import GaussianNB

In [41]:
gnb = GaussianNB()

In [42]:
gnb.fit(train_x.toarray(), train_labels.ravel())

GaussianNB()

In [43]:
gnb.score(test_x.toarray(), test_labels.ravel())

0.54032258064516125

### Support Vector Classifier

In [44]:
from sklearn.svm import SVC

In [45]:
supportvc = SVC()

In [46]:
supportvc.fit(train_x.toarray(), train_labels.ravel())

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [47]:
supportvc.score(test_x.toarray(), test_labels.ravel())

0.5161290322580645